# Classification model - Saving Pickle File

In [1]:
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

### Load the data

In [2]:
data = pd.read_csv('SBALoanData.csv')
print(f"Shape of the data is: {data.shape}")
data.head()

Shape of the data is: (2465, 15)


,SBA Physical Declaration Number,SBA EIDL Declaration Number,FEMA Disaster Number,SBA Disaster Number,Damaged Property City Name,Damaged Property Zip Code,Damaged Property County/Parish Name,Damaged Property State Code,Total Verified Loss,Verified Loss Real Estate,Verified Loss Content,Total Approved Loan Amount,Approved Amount Real Estate,Approved Amount Content,LoanApproved
0,15742,15743,4399.0,FL-00140,ALFORD,32420,JACKSON,FL,3744250.0,3262237.0,482013.0,1944800.0,1668200.0,276600.0,YES
1,15742,15743,4399.0,FL-00140,ALFORD,32420,WASHINGTON,FL,178062.0,137332.0,40730.0,119100.0,94400.0,24700.0,YES
2,15742,15743,4399.0,FL-00140,ALLIGATOR POINT,32346,FRANKLIN,FL,513784.0,447859.0,65925.0,106600.0,93700.0,12900.0,YES
3,15742,15743,4399.0,FL-00140,ALTHA,32421,CALHOUN,FL,17594222.0,14952549.0,2641673.0,9947400.0,8478800.0,1468600.0,YES
4,15742,15743,4399.0,FL-00140,APALACHICOLA,32320,FRANKLIN,FL,1812552.0,1576667.0,235885.0,941900.0,835100.0,106800.0,YES


In [3]:
# ## getting top 30% of the data

# counts = data['Lender'].value_counts()

# top_data = data.loc[data['Lender'].isin(counts.index[counts > 5])]
# top_data.head()

### Check how many `null/Nan` values are in each column

In [4]:
data.isnull().sum()

SBA Physical Declaration Number           0
SBA EIDL Declaration Number               0
FEMA Disaster Number                    960
SBA Disaster Number                       0
Damaged Property City Name                0
Damaged Property Zip Code                 0
Damaged Property County/Parish Name       0
Damaged Property State Code               0
Total Verified Loss                    2119
Verified Loss Real Estate               516
Verified Loss Content                   516
Total Approved Loan Amount                1
Approved Amount Real Estate             516
Approved Amount Content                 516
LoanApproved                              0
dtype: int64

### View the possible labels for the columns that have some null values

In [5]:
# # we are intentionally narrowing the scope of the loan data details for those with strong completion entries, minimal nulls
# contains_null = ['Zip', 'BusinessType', 'DateApproved', 'Lender','CD', 'LoanRange']

# for col in contains_null:
#     print(f"List of unique labels for {col}:::{set(data[col])}")

### Clean up null values

For the sake of simplicity, I'm going to make a few assumptions:

- `Dependents`: Assumption that there are no dependents (0: 345 | 1: 102 | 2: 101 | 3+: 51)
- `Self_Employed`: Assumption that the applicant is not self-employed (No: 500 | Yes: 82)
- `Credit_History`: Assumption that the person has a credit history (True: 475 | False: 89)
- `Married`: If nothing specified, applicant is not married
- `Gender`: Assuming the gender is Male for the missing values (Male: 489 | Female: 112)

In [6]:
# data['Dependents'] = data['Dependents'].fillna('0')
# data['Self_Employed'] = data['Self_Employed'].fillna('No')
# data['Credit_History'] = data['Credit_History'].fillna(1)
# data['Married'] = data['Married'].fillna('No')
# data['Gender'] = data['Gender'].fillna('Male')

In [7]:
# investigate rows with null values
data[data.isnull().any(axis=1)]

,SBA Physical Declaration Number,SBA EIDL Declaration Number,FEMA Disaster Number,SBA Disaster Number,Damaged Property City Name,Damaged Property Zip Code,Damaged Property County/Parish Name,Damaged Property State Code,Total Verified Loss,Verified Loss Real Estate,Verified Loss Content,Total Approved Loan Amount,Approved Amount Real Estate,Approved Amount Content,LoanApproved
272,15800,15801,NaN,HI-00051,HILO,96720,HAWAII,HI,168410.0,98810.00,69600.0,125800.0,85800.0,40000.0,YES
273,15800,15801,NaN,HI-00051,HILO,96721,HAWAII,HI,1.0,1.00,0.0,0.0,0.0,0.0,NO
274,15800,15801,NaN,HI-00051,KEAAU,96749,HAWAII,HI,494271.0,335071.00,159200.0,415300.0,335300.0,80000.0,YES
275,15800,15801,NaN,HI-00051,KURTISTOWN,96760,HAWAII,HI,10892.0,10892.00,0.0,10900.0,10900.0,0.0,YES
276,15800,15801,NaN,HI-00051,LAHAINA,96761,MAUI,HI,2356258.0,1972658.00,383600.0,40000.0,0.0,40000.0,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,16141,16142,NaN,IL-00056,UNIVERSITY PARK,60484,Will,IL,NaN,21729.00,18075.0,0.0,0.0,0.0,NO
2461,16141,16142,NaN,IL-00056,AURORA,60506,Will,IL,NaN,NaN,NaN,0.0,NaN,NaN,NO
2462,16141,16142,NaN,IL-00056,CHICAGO,60621,Cook,IL,NaN,0.00,4565.0,0.0,0.0,0.0,NO
2463,16141,16142,NaN,IL-00056,CHICAGO,60628,Cook,IL,NaN,37469.66,7300.0,44800.0,37500.0,7300.0,YES


In [10]:
# only take rows of businesses that CD is not null

# data = data[data['Total Verified Loss'].notna()]

data = data[data['FEMA Disaster Number'].notna()]
# data.head()

# # #drop rows with NA cities
# # data = data[data['City'].notna()]

# # # drop rows with NA NAICS code
# # data = data[data['NAICSCode'].notna()]

# # #drop rows with NA business type
# # data = data[data['BusinessType'].notna()]

# # data.isnull().sum()

# # data.shape

# # data.head()

# # data['NAICSCode'] = data['NAICSCode'].astype(int)

# data

# data.isnull().sum()

# data.head()

data.shape

(1505, 15)

### View cleaned up values

In [11]:
# label_cols = ['LoanAmount', 'City', 'State', 'Zip', 'NAICSCode', 'BusinessType', 'DateApproved', 'Lender', 'CD', 'LoanRange']

# for col in label_cols:
#     print(f"List of unique labels for {col} ::: {set(data[col])}")

In [12]:
## read in data for NAICS code

# naics = pd.read_csv('US-Business-Profiles-By-Sales-and-Employees (2).csv')

# naics.head()


In [13]:
# naics.dtypes
# naics[naics.isnull().any(axis=1)]


# naics['NAICS 1 Code'] = naics['NAICS 1 Code'].astype(int)

# naics

In [14]:
## merge tables 

# pd.merge(data, naics, left_on='NAICSCode', right_on='NAICS 1 Code')

### Encode categorical fields
We have a lot of `string` labels that we encounter in `Gender`, `Married`, `Education`, `Self_Employed` & `Property_Area` columns.

In [15]:
data.head()

,SBA Physical Declaration Number,SBA EIDL Declaration Number,FEMA Disaster Number,SBA Disaster Number,Damaged Property City Name,Damaged Property Zip Code,Damaged Property County/Parish Name,Damaged Property State Code,Total Verified Loss,Verified Loss Real Estate,Verified Loss Content,Total Approved Loan Amount,Approved Amount Real Estate,Approved Amount Content,LoanApproved
0,15742,15743,4399.0,FL-00140,ALFORD,32420,JACKSON,FL,3744250.0,3262237.0,482013.0,1944800.0,1668200.0,276600.0,YES
1,15742,15743,4399.0,FL-00140,ALFORD,32420,WASHINGTON,FL,178062.0,137332.0,40730.0,119100.0,94400.0,24700.0,YES
2,15742,15743,4399.0,FL-00140,ALLIGATOR POINT,32346,FRANKLIN,FL,513784.0,447859.0,65925.0,106600.0,93700.0,12900.0,YES
3,15742,15743,4399.0,FL-00140,ALTHA,32421,CALHOUN,FL,17594222.0,14952549.0,2641673.0,9947400.0,8478800.0,1468600.0,YES
4,15742,15743,4399.0,FL-00140,APALACHICOLA,32320,FRANKLIN,FL,1812552.0,1576667.0,235885.0,941900.0,835100.0,106800.0,YES


In [16]:
# create dictionaries to map fields to numeric values
# gender_values = {'Female' : 0, 'Male' : 1} 
# married_values = {'No' : 0, 'Yes' : 1}
# education_values = {'Graduate' : 0, 'Not Graduate' : 1}
# employed_values = {'No' : 0, 'Yes' : 1}
# dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
# target_values = {'Y':1, 'N':0}
loan_approved = {'NO' : 0, 'YES' : 1}

# # replace values in each column according to the dictionaries above
# data.replace({'Gender': gender_values, 'Married': married_values, 'Education': education_values, \
#                 'Self_Employed': employed_values, 'Dependents': dependent_values, 'Loan_Status': target_values}, inplace=True)

data.replace({'LoanApproved': loan_approved}, inplace=True)

# Get dummy variables for nominal property column
# clean_data = pd.get_dummies(data, columns=["BusinessType"])

data.head()

,SBA Physical Declaration Number,SBA EIDL Declaration Number,FEMA Disaster Number,SBA Disaster Number,Damaged Property City Name,Damaged Property Zip Code,Damaged Property County/Parish Name,Damaged Property State Code,Total Verified Loss,Verified Loss Real Estate,Verified Loss Content,Total Approved Loan Amount,Approved Amount Real Estate,Approved Amount Content,LoanApproved
0,15742,15743,4399.0,FL-00140,ALFORD,32420,JACKSON,FL,3744250.0,3262237.0,482013.0,1944800.0,1668200.0,276600.0,1
1,15742,15743,4399.0,FL-00140,ALFORD,32420,WASHINGTON,FL,178062.0,137332.0,40730.0,119100.0,94400.0,24700.0,1
2,15742,15743,4399.0,FL-00140,ALLIGATOR POINT,32346,FRANKLIN,FL,513784.0,447859.0,65925.0,106600.0,93700.0,12900.0,1
3,15742,15743,4399.0,FL-00140,ALTHA,32421,CALHOUN,FL,17594222.0,14952549.0,2641673.0,9947400.0,8478800.0,1468600.0,1
4,15742,15743,4399.0,FL-00140,APALACHICOLA,32320,FRANKLIN,FL,1812552.0,1576667.0,235885.0,941900.0,835100.0,106800.0,1


### Train/Test Split

In [17]:
# store the target variable in y and everything else goes in X
y = data['LoanApproved']

# we're also dropping Load_ID because it adds no value to the prediction
X = data.drop(['SBA Disaster Number', 'Damaged Property City Name', 'Damaged Property County/Parish Name', 'Damaged Property State Code'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

X_train.shape

(1128, 11)

### Fill the nulls for the continuous variables with the mean values of that column

In [18]:
# X_train['Loan_Amount_Term'] = X_train['Loan_Amount_Term'].fillna(X_train['Loan_Amount_Term'].mean())
# X_train['LoanAmount'] = X_train['LoanAmount'].fillna(X_train['LoanAmount'].mean())
# X_test['Loan_Amount_Term'] = X_test['Loan_Amount_Term'].fillna(X_test['Loan_Amount_Term'].mean())
# X_test['LoanAmount'] = X_test['LoanAmount'].fillna(X_test['LoanAmount'].mean())


# # View the datatypes of all columns
# X_train.dtypes

### Confirm that we no longer have any nulls

In [19]:
X_train.isnull().sum()

SBA Physical Declaration Number      0
SBA EIDL Declaration Number          0
FEMA Disaster Number                 0
Damaged Property Zip Code            0
Total Verified Loss                932
Verified Loss Real Estate          217
Verified Loss Content              217
Total Approved Loan Amount           1
Approved Amount Real Estate        217
Approved Amount Content            217
LoanApproved                         0
dtype: int64

### Create the classifier model and the parameter grid for GridSearch

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

param_grid = {"n_estimators" : [10, 20, 50, 100],
             "max_depth" : [None, 6, 8, 10],
             "max_leaf_nodes": [None, 5, 10, 20], 
             "min_impurity_split": [0.1, 0.2, 0.3, 0.4]}

### Initialize the GridSearch to tune my hyperparameters

In [14]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(model, param_grid=param_grid, cv=3, verbose=2)

### Execute the tuning

In [15]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 256 candidates, totalling 768 fits
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=10, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=10, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=10, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=20, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=None, min_impurity_split=0.1, n_estimators=20, total=   0.0s
[CV] max_depth=None, max_leaf_no

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_split=0.3, n_estimators=100, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=10, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=10, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=10 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=10, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=20, total=   0.0s
[CV] max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=20 
[CV]  max_depth=None, max_leaf_nodes=5, min_impurity_split=0.4, n_estimators=20, total=   0.0s
[CV] max_depth=None, max_leaf

[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    1.8s finished


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

### View the results

In [21]:
print(f"Best parameters: {grid.best_params_}")
print(f"Test set score: {grid.score(X_test, y_test)}")

Best parameters: {'max_depth': None, 'max_leaf_nodes': None, 'min_impurity_split': 0.1, 'n_estimators': 100}
Test set score: 1.0


# Saving the model

In [ ]:
import pickle

with open('RF_model.pkl','wb') as f:
    pickle.dump(grid, f)

### Loading the saved model

In [ ]:
with open('RF_model.pkl', 'rb') as f:
    model = pickle.load(f)

### Test the saved model

In [ ]:
print(f"Test set score: {model.score(X_test, y_test)}")

In [ ]:
model